<a href="https://colab.research.google.com/github/yuuki-kusumoto/kusumoto/blob/main/bert_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Nov  8 18:49:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install MeCab
!pip install mecab-python3
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

In [ ]:
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')

In [ ]:
!pip install ipadic
!pip install mecab-python3
!pip install unidic-lite
# MeCabとtransformersを用意する
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
# 以下で報告があるようにmecab-python3のバージョンを0.996.5にしないとtokezerで落ちる
# https://stackoverflow.com/questions/62860717/huggingface-for-japanese-tokenizer
!pip install mecab-python3==0.996.5
!pip install unidic-lite # これないとMeCab実行時にエラーで落ちる
!pip install transformers
!pip install "transformers==2.5.1"

In [ ]:
import torch
import transformers
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
from transformers import BertJapaneseTokenizer, BertForMaskedLM

# パッケージのimport
import numpy as np
import random
import torch.nn as nn
import torch.optim as optim
import torchtext

# 必要なパッケージのimport
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext


! curl http://www.cl.ecei.tohoku.ac.jp/resources/sent_lex/pn.csv.m3.120408.trim > pn.csv


model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# model_nameはここから取得(cf. https://huggingface.co/transformers/pretrained_models.html)
model_name = "cl-tohoku/bert-base-japanese"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

In [ ]:
# ファイルをオープンして、1行ずつその内容を読み込んで処理する
with open('train_data.txt') as f:
    for line in f:
        line = line.rstrip()  # 読み込んだ行の末尾には改行文字があるので削除
        print(line)
        
with open('train_data.txt') as f:
  texts = f.read()

#訓練データをリストにして開く

with open('train_data.txt', 'r') as f:
    tr_list = f.read().split("\n")

#訓練データをラベルと文章に分けてリストに入れる。

tr_word = []
tr_label = []

for i in range(0,4000):
  a = tr_list[i]
  b = a.replace('\t','')

  tr_word.append(b[1:len(b)])
  tr_label.append(b[0])

print(tr_word)
print(tr_label)

In [ ]:
with open('test_data.txt') as f:
  texts = f.read()
  
#テストデータをリストにして開く

with open('test_data.txt', 'r') as f:
    ts_list = f.read().split("\n")

#テストデータをラベルと文章に分けてリストに入れる。

ts_word = []
ts_label = []

for i in range(0,2000):
  a = ts_list[i]
  b = a.replace('\t','')

  ts_word.append(b[1:len(b)])
  ts_label.append(b[0])

print(ts_word)
print(ts_label)

In [ ]:
#フレームに
import pandas as pd

#text, indexの順番にする
tr_df1 = pd.DataFrame({'tr_word' : tr_word, 'tr_label' : tr_label})
print(len(tr_df1))

#フレームに
import pandas as pd

ts_df = pd.DataFrame({'tr_word' : ts_word, 'tr_label' : ts_label})
print(len(ts_df))

tr_df = pd.concat([tr_df1, ts_df])
print(len(tr_df))
tr_df = tr_df.sample(frac=1, random_state=123).reset_index(drop=True)
tr_df.shape

In [ ]:
# tsvファイルで保存する
#tr_df = tr_df[:100]
# 全体の2割の文章数
len_0_2 = len(tr_df) // 5

# 前から2割をテストデータとする
tr_df[:len_0_2].to_csv("./test.tsv", sep='\t', index=False, header=None)
print(tr_df[:len_0_2].shape)

# 前2割からを訓練&検証データとする
tr_df[len_0_2:].to_csv("./train.tsv", sep='\t', index=False, header=None)
print(tr_df[len_0_2:].shape)

In [ ]:

# torchtext.data.Datasetのsplit関数で訓練データと検証データを分ける
# train_eval：4000個、test：2000個

dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio = 1 - 0.25, random_state=random.seed(1234))

# datasetの長さを確認してみる
print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())

In [ ]:
# 乱数シードの固定

import os
import random
import numpy as np
import torch

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合

# torchtext.data.Datasetのsplit関数で訓練データと検証データを分ける
# train_eval：4000個、test：2000個

dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio = 1 - 0.25, random_state=random.seed(1234))

# datasetの長さを確認してみる
print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())

# datasetの中身を確認してみる
item = next(iter(dataset_train))
print(item.Text)
print("長さ：", len(item.Text))  # 長さを確認 [CLS]から始まり[SEP]で終わる。512より長いと後ろが切れる
print("ラベル：", item.Label)

In [ ]:
# DataLoaderを作成

batch_size = 128

dl_train = torchtext.legacy.data.Iterator(
    dataset_train, batch_size=batch_size, train=True)

dl_eval = torchtext.legacy.data.Iterator(
    dataset_eval, batch_size=batch_size, train=False, sort=False)

dl_test = torchtext.legacy.data.Iterator(
    dataset_test, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dl_train, "val": dl_eval}

In [ ]:
from torch import nn

class BertForReview(nn.Module):
    '''BERTモデルにレビュー文の2クラスを判定する部分をつなげたモデル'''

    def __init__(self):
        super(BertForReview, self).__init__()

        # BERTモジュール
        self.bert = model  # 日本語学習済みのBERTモデル

        # headにポジネガ予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は14クラス
        #cls層の追加
        self.cls = nn.Linear(in_features=768, out_features=2)

        
        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)
    
    
    def forward(self, input_ids):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        #print("input_ids :", input_ids)
        result = self.bert(input_ids)  # reult は、sequence_output, pooled_output

        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]  # 最初の0がsequence_outputを示す
        vec_02 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素。　つまりclsトークンを獲得している。
        vec_02 = vec_02.view(-1, 768)  # sizeを[batch_size, hidden_size]
        vec_03 = vec_0[:, 0:512, :]
        output = self.cls(vec_02)  # 全結合層
        m = nn.Sigmoid()
        out = m(output)

        return vec_02, vec_03

#モデル構築
BERT_net = BertForReview()

print('ネットワーク設定完了')

# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for param in BERT_net.parameters():
    param.requires_grad = False

# 2. BertLayerモジュールの最後を勾配計算ありに変更
for param in BERT_net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for param in BERT_net.cls.parameters():
    param.requires_grad = True

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext

class PositionalEncoder(nn.Module):
    '''入力された単語の位置を示すベクトル情報を付加する'''

#max_seq_len はパディングしないといけない？
    def __init__(self, d_model=768, max_seq_len=512):
        super().__init__()

        self.d_model = d_model  # 単語ベクトルの次元数

        # 単語の順番（pos）と埋め込みベクトルの次元の位置（i）によって一意に定まる値の表をpeとして作成
        pe = torch.zeros(max_seq_len, d_model)
        #print(pe)

        # GPUが使える場合はGPUへ送る
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        pe = pe.to(device)

        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos /
                                          (10000 ** ((2 * (i + 1))/d_model)))

        # 表peの先頭に、ミニバッチ次元となる次元を足す
        self.pe = pe.unsqueeze(0)

        # 勾配を計算しないようにする
        self.pe.requires_grad = False

    def forward(self, x):

        # 入力xとPositonal Encodingを足し算する
        # xがpeよりも小さいので、大きくする
        ret = math.sqrt(self.d_model)*x + self.pe
        return ret


class Attention(nn.Module):
    '''シングルAttentionで実装'''

    def __init__(self, d_model=768):
        super().__init__()

        # SAGANでは1dConvを使用したが、今回は全結合層で特徴量を変換する
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)

        # 出力時に使用する全結合層
        self.out = nn.Linear(d_model, d_model)

        # Attentionの大きさ調整の変数
        self.d_k = d_model

    def forward(self, cls_tokens, pe):
      #mask = torch.zeros(768)
      #for i in range(0, len(cls_tokens)):
        
        # 全結合層で特徴量を変換
        k = self.k_linear(pe)
        q = self.q_linear(cls_tokens)
        v = self.v_linear(pe)

        # Attentionの値を計算する
        # 各値を足し算すると大きくなりすぎるので、root(d_k)で割って調整
        
        #attention_weights = torch.matmul(q, k.transpose(2, 1)) / math.sqrt(self.d_k)

        attention_weights = torch.matmul(q.unsqueeze(1), k.transpose(2, 1) )/ math.sqrt(self.d_k)



        # ここでmaskを計算
        #mask = mask.unsqueeze(1)
        #attention_weights = attention_weights.masked_fill(mask == 0, -1e9)

        # softmaxで規格化をする
        normlized_weights = F.softmax(attention_weights, dim=-1)

        # AttentionをValueとかけ算
        output = torch.matmul(normlized_weights, v)

        # 全結合層で特徴量を変換
        output = self.out(output)
        

        return output, normlized_weights


class FeedForward(nn.Module):
    def __init__(self, d_model = 768, d_ff=1024, dropout=0.1):
        '''Attention層から出力を単純に全結合層2つで特徴量を変換するだけのユニットです'''
        super().__init__()

        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.dropout(F.relu(x))
        x = self.linear_2(x)
        return x

class TransformerBlock(nn.Module):
    def __init__(self, d_model=768, dropout=0.1):
        super().__init__()

        # LayerNormalization層
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)

        # Attention層
        self.attn = Attention(d_model)

        # Attentionのあとの全結合層2つ
        self.ff = FeedForward(d_model)

        # Dropout
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def forward(self, pn, x):
        # PositionalEncoding を行った後に正規化する
        x_normlized = self.norm_1(x)

        output, normlized_weights = self.attn(pn, x_normlized)
        #FeedForward層の入力作成
        x2 = x + self.dropout_1(output)

        # 正規化と全結合層
        x_normlized2 = self.norm_2(x2)
        output = x2 + self.dropout_2(self.ff(x_normlized2))

        return output, normlized_weights

class ClassificationHead(nn.Module):
    '''Transformer_Blockの出力を使用し、最後にクラス分類させる'''

    def __init__(self, d_model=768, output_dim=2):
        super().__init__()

        # 全結合層
        self.linear = nn.Linear(d_model, output_dim)  # output_dimはポジ・ネガの2つ

        # 重み初期化処理
        nn.init.normal_(self.linear.weight, std=0.02)
        nn.init.normal_(self.linear.bias, 0)

    def forward(self, x):
        x0 = x[:, 0, :]  # 各ミニバッチの各文の cls の特徴量（768次元）を取り出す
        out = self.linear(x0)

        return out
        
# 最終的なTransformerモデルのクラス


class TransformerClassification(nn.Module):
    '''Transformerでクラス分類させる'''

    def __init__(self,  d_model=768, max_seq_len=512, output_dim=2):
        super().__init__()

        # モデル構築
        
        self.net1 = BERT_net
        self.net2 = PositionalEncoder(d_model=d_model, max_seq_len=max_seq_len)
        self.net3_1 = TransformerBlock(d_model=d_model)
        #self.net3_2 = TransformerBlock(d_model=d_model)
        self.net4 = ClassificationHead(output_dim=output_dim, d_model=d_model)

    def forward(self, text):

        cls, x1 = self.net1(text)

        x2 = self.net2(x1)  # Positon情報を足し算
        
        x3_1, normlized_weights_1 = self.net3_1(cls, x2)  # Self-Attentionで特徴量を変換
        
        #x3_2, normlized_weights_2 = self.net3_2(x3_1, mask)  # Self-Attentionで特徴量を変換
        
        x4 = self.net4(x3_1)  # 最終出力の0単語目を使用して、分類0-1のスカラーを出力
        
        return x4, normlized_weights_1 #, normlized_weights_2

In [ ]:
# モデル構築
TC_net = TransformerClassification()
    
# ネットワークの初期化を定義
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        # Liner層の初期化
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

# 訓練モードに設定
TC_net.train()

# TransformerBlockモジュールを初期化実行
TC_net.net3_1.apply(weights_init)
#TC_net.net3_2.apply(weights_init)
print('ネットワーク設定完了')

# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

# 最適化手法の設定
learning_rate = 2e-5
optimizer = optim.Adam(TC_net.parameters(), lr=learning_rate)

In [ ]:
# モデルを学習させる関数を作成

def train_model(tc_net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')
    # ネットワークをGPUへ
    #bert_net.to(device)
    tc_net.to(device)

    train_loss = []
    val_loss = []

    acc_train_list = []
    acc_val_list = []



    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    for epoch in range(num_epochs):

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:

            if phase == 'train':
                #bert_net.train()  # モデルを訓練モードに
                tc_net.train()
            else:
                #bert_net.eval()   # モデルを検証モードに
                tc_net.eval()


            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書オブジェクト

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                #BERTに入力
                #cls_tokens, bert_tokens  = bert_net(inputs)


                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):


                    # Transformerに入力
                    outputs, weights  = tc_net(inputs)
                    
                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測（dim=1 列方向のＭａｘを取得、predsは最大のindex）

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()   #損失の計算
                        optimizer.step()  # 勾配の更新


                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            if phase == 'train':
              train_loss = np.append(train_loss, epoch_loss)
              acc_train_list = np.append(acc_train_list, epoch_acc.cpu().numpy())

            else:
              val_loss = np.append(val_loss, epoch_loss)
              acc_val_list = np.append(acc_val_list, epoch_acc.cpu().numpy())

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))


    return tc_net, train_loss, val_loss, acc_train_list, acc_val_list
